In [9]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [10]:
df = pd.read_csv("../dataset/daily_food_nutrition_dataset.csv")

In [11]:
features = ["Calories (kcal)", "Protein (g)", "Carbohydrates (g)", "Fat (g)", "Fiber (g)", "Sugars (g)", "Sodium (mg)", "Cholesterol (mg)"]
X = df[features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

knn = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn.fit(X_scaled)


NearestNeighbors(metric='euclidean', n_neighbors=6)

In [ ]:
def generate_meal_plan(start_food, days=30):
    meal_plan = [start_food]
    current_food = start_food
    visited = set()
    visited.add(current_food.lower())

    for _ in range(days - 1):
        idx = df[df["Food_Item"].str.lower() == current_food.lower()].index
        if len(idx) == 0:
            meal_plan.append("Not Found")
            break
        
        idx = idx[0]
        distances, indices = knn.kneighbors([X_scaled[idx]])
        
        # Find first unvisited similar food
        found = False
        for neighbor_idx in indices[0][1:]:
            neighbor_food = df.iloc[neighbor_idx]['Food_Item']
            if neighbor_food.lower() not in visited:
                meal_plan.append(neighbor_food)
                current_food = neighbor_food
                visited.add(neighbor_food.lower())
                found = True
                break
        
        if not found:
            # If stuck, repeat last food
            meal_plan.append(current_food)
    
    # Ensure exactly 30 foods
    while len(meal_plan) < days:
        meal_plan.append(meal_plan[-1])
    
    return meal_plan

In [13]:
# Starting foods
firstMealDf = pd.read_csv("results/first_meal_plan.csv")
start_foods = firstMealDf["Food_Item"].tolist()

In [ ]:
date_to_target = int(input("Type the duration you want to follow plans: "))

In [ ]:
all_meal_plans = {}

for food in start_foods:
    plan = generate_meal_plan(food, date_to_target)
    all_meal_plans[food] = plan

# Create final DataFrame
meal_plans_df = pd.DataFrame(all_meal_plans).T
meal_plans_df.columns = [f"Day {i+1}" for i in range(date_to_target)]

# Show the final table
print(meal_plans_df.to_string(index=True))

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid